start


In [1]:
import numpy as np
import pandas as pd


from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score

In [2]:
# Load the data
data_path = './datasets/group47/dataset/intermediate_datafiles/chapter5_group47_result.csv'
try:
    data = pd.read_csv(data_path, index_col=0)
except FileNotFoundError:
    print(f"File not found at {data_path}")
    exit()

In [3]:
# Define the target columns
target_columns = ['labelCycling', 'labelStairs', 'labelWalking', 'labelSitting', 'labelOther']

In [4]:
# Separate the features and target variables
X = data.drop(target_columns, axis=1)
y = data[target_columns]

# Fill NaN values with 0 (or any other value you deem appropriate)
X = X.fillna(0)

# Replace positive infinity values with the maximum non-infinity number
X = X.replace([np.inf], np.finfo('float64').max)

# Replace negative infinity values with the minimum non-infinity number
X = X.replace([-np.inf], np.finfo('float64').min)

In [5]:
selected_features = set()
for column in y.columns:
    y_single_label = LabelEncoder().fit_transform(y[column])  # Convert to 1D array
    selector = SelectKBest(f_classif, k=50)
    X_new = selector.fit_transform(X, y_single_label)
    
    # Get the selected features for this label
    mask = selector.get_support()  # List of booleans for selected features
    selected_features_single_label = X.columns[mask]
    
    # Add the selected features for this label to the set of all selected features
    selected_features.update(selected_features_single_label)

# Filter the features based on the selected features
X = X[list(selected_features)]

c:\Users\patri\.conda\envs\ml4qs_p3_8\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: UserWarning: Features [48 96] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx,
c:\Users\patri\.conda\envs\ml4qs_p3_8\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw
c:\Users\patri\.conda\envs\ml4qs_p3_8\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
c:\Users\patri\.conda\envs\ml4qs_p3_8\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: UserWarning: Features [49 54 96] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx,
c:\Users\patri\.conda\envs\ml4qs_p3_8\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  f = 

In [6]:
# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

 # Use simple random sampling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [7]:
# Define the LSTM model
model = Sequential()
model.add(LSTM(200, return_sequences=True, input_shape=(X_train.shape[1], 1)))  # Increase the number of units
model.add(Dropout(0.2))
model.add(LSTM(200, return_sequences=True))  # Add another LSTM layer
model.add(Dropout(0.2))
model.add(LSTM(100))  # Keep this LSTM layer
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu'))
model.add(Dense(len(target_columns), activation='sigmoid'))  

In [9]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])  # Use learning_rate instead of lr

In [10]:
# Make predictions on the testing data
y_pred = model.predict(X_test)

38/38 [==============================] - 3s 46ms/step


In [11]:
# Convert the predictions to binary format
y_pred_bin = (y_pred > 0.5).astype(int)

In [12]:
# Print the classification report
print(classification_report(y_test, y_pred_bin, target_names=target_columns))
precision = precision_score(y_test, y_pred_bin, average='weighted', zero_division=1)
recall = recall_score(y_test, y_pred_bin, average='weighted', zero_division=1)
f1 = f1_score(y_test, y_pred_bin, average='weighted', zero_division=1)


              precision    recall  f1-score   support

labelCycling       0.24      0.66      0.35       247
 labelStairs       0.09      0.09      0.09        64
labelWalking       0.09      0.04      0.06       192
labelSitting       0.64      0.52      0.58       649
  labelOther       0.00      0.00      0.00        50

   micro avg       0.34      0.43      0.38      1202
   macro avg       0.21      0.26      0.21      1202
weighted avg       0.42      0.43      0.40      1202
 samples avg       0.34      0.43      0.36      1202



c:\Users\patri\.conda\envs\ml4qs_p3_8\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\patri\.conda\envs\ml4qs_p3_8\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
# Print precision, recall, and F1 score
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


Precision: 0.4163865660747246
Recall: 0.42678868552412647
F1 Score: 0.39620574487846194


In [14]:
# Define your classifier
clf = RandomForestClassifier(random_state=42, class_weight='balanced')

# Wrap your classifier with OneVsRestClassifier
clf = OneVsRestClassifier(clf)


In [15]:
# Define your parameter grid
param_grid = {
    'estimator__n_estimators': [50, 100, 200],
    'estimator__max_features': ['sqrt', 'log2'],
    'estimator__max_depth' : [4,5,6,7,8],
    'estimator__criterion' :['gini', 'entropy']
}

In [16]:
# Initialize GridSearchCV with F1 score as the scoring metric
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='f1_macro')  # Use F1 score

# Fit the model
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=OneVsRestClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                            ccp_alpha=0.0,
                                                                            class_weight='balanced',
                                                                            criterion='gini',
                                                                            max_depth=None,
                                                                            max_features='auto',
                                                                            max_leaf_nodes=None,
                                                                            max_samples=None,
                                                                            min_impurity_decrease=0.0,
                                                                            min_impurity_split=None,
  

In [17]:
# Get the best parameters
best_params = grid_search.best_params_

print(f"Best parameters: {best_params}")

Best parameters: {'estimator__criterion': 'entropy', 'estimator__max_depth': 4, 'estimator__max_features': 'log2', 'estimator__n_estimators': 50}


In [20]:
# Define the LSTM model
model = Sequential()
model.add(LSTM(200, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, input_shape=(X_train.shape[1], 1)))  # Add dropout and recurrent dropout
model.add(LSTM(100, return_sequences=False, dropout=0.2, recurrent_dropout=0.2))  # Reduce the number of units and add dropout and recurrent dropout
model.add(Dense(10, activation='relu'))
model.add(Dense(len(target_columns), activation='sigmoid')) 

# Compile the model
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])  # Use learning_rate instead of lr

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)  # Stop training when the validation loss has not improved for 10 epochs

# Fit the model with the training data
model.fit(X_train, y_train, validation_split=0.2, epochs=30, callbacks=[early_stopping])  # Add validation split and early stopping

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Convert the predictions to binary format
y_pred_bin = (y_pred > 0.5).astype(int)

# Print the classification report
print(classification_report(y_test, y_pred_bin, target_names=target_columns))

# Calculate precision, recall, and F1 score
precision = precision_score(y_test, y_pred_bin, average='weighted', zero_division=1)
recall = recall_score(y_test, y_pred_bin, average='weighted', zero_division=1)
f1 = f1_score(y_test, y_pred_bin, average='weighted', zero_division=1)

# Print precision, recall, and F1 score
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


Epoch 1/100
61/61 [==============================] - 15s 191ms/step - loss: 0.4196 - accuracy: 0.5477 - val_loss: 0.3631 - val_accuracy: 0.5700
Epoch 2/100
61/61 [==============================] - 11s 182ms/step - loss: 0.3658 - accuracy: 0.5770 - val_loss: 0.3532 - val_accuracy: 0.6008
Epoch 3/100
61/61 [==============================] - 11s 184ms/step - loss: 0.3550 - accuracy: 0.5884 - val_loss: 0.3200 - val_accuracy: 0.6893
Epoch 4/100
61/61 [==============================] - 11s 179ms/step - loss: 0.2947 - accuracy: 0.7105 - val_loss: 0.2558 - val_accuracy: 0.7737
Epoch 5/100
61/61 [==============================] - 11s 180ms/step - loss: 0.2449 - accuracy: 0.7599 - val_loss: 0.2278 - val_accuracy: 0.7737
Epoch 6/100
61/61 [==============================] - 11s 181ms/step - loss: 0.2419 - accuracy: 0.7414 - val_loss: 0.2273 - val_accuracy: 0.7510
Epoch 7/100
61/61 [==============================] - 11s 179ms/step - loss: 0.2429 - accuracy: 0.7110 - val_loss: 0.2260 - val_accuracy:

c:\Users\patri\.conda\envs\ml4qs_p3_8\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
# Define the LSTM model
model = Sequential()
model.add(LSTM(200, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, input_shape=(X_train.shape[1], 1)))  # Add dropout and recurrent dropout
model.add(LSTM(100, return_sequences=False, dropout=0.2, recurrent_dropout=0.2))  # Reduce the number of units and add dropout and recurrent dropout
model.add(Dense(10, activation='relu'))
model.add(Dense(len(target_columns), activation='sigmoid'))  # Change activation function to softmax

# Compile the model
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])  # Use learning_rate instead of lr

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)  # Stop training when the validation loss has not improved for 10 epochs

# Fit the model with the training data
model.fit(X_train, y_train, validation_split=0.2, epochs=100, callbacks=[early_stopping])  # Add validation split and early stopping

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Convert the predictions to binary format
y_pred_bin = (y_pred > 0.5).astype(int)

# Print the classification report
print(classification_report(y_test, y_pred_bin, target_names=target_columns))

# Calculate precision, recall, and F1 score
precision = precision_score(y_test, y_pred_bin, average='weighted', zero_division=1)
recall = recall_score(y_test, y_pred_bin, average='weighted', zero_division=1)
f1 = f1_score(y_test, y_pred_bin, average='weighted', zero_division=1)

# Print precision, recall, and F1 score
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


Epoch 1/100
61/61 [==============================] - 14s 179ms/step - loss: 0.4832 - accuracy: 0.4894 - val_loss: 0.4041 - val_accuracy: 0.5288
Epoch 2/100
61/61 [==============================] - 12s 198ms/step - loss: 0.3985 - accuracy: 0.5183 - val_loss: 0.3718 - val_accuracy: 0.5288
Epoch 3/100
61/61 [==============================] - 12s 199ms/step - loss: 0.3836 - accuracy: 0.5188 - val_loss: 0.3659 - val_accuracy: 0.5288
Epoch 4/100
61/61 [==============================] - 13s 206ms/step - loss: 0.3707 - accuracy: 0.5245 - val_loss: 0.3481 - val_accuracy: 0.5288
Epoch 5/100
61/61 [==============================] - 12s 203ms/step - loss: 0.3378 - accuracy: 0.5873 - val_loss: 0.2983 - val_accuracy: 0.7058
Epoch 6/100
61/61 [==============================] - 13s 207ms/step - loss: 0.2799 - accuracy: 0.7043 - val_loss: 0.2461 - val_accuracy: 0.7058
Epoch 7/100
61/61 [==============================] - 13s 213ms/step - loss: 0.2495 - accuracy: 0.7038 - val_loss: 0.2341 - val_accuracy:

c:\Users\patri\.conda\envs\ml4qs_p3_8\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\patri\.conda\envs\ml4qs_p3_8\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
from keras.regularizers import l1_l2

# Define the LSTM model
model = Sequential()
model.add(LSTM(200, return_sequences=True, dropout=0.3, recurrent_dropout=0.3, 
               kernel_regularizer=l1_l2(l1=0.01, l2=0.01),  # Add L1/L2 regularization
               input_shape=(X_train.shape[1], 1)))
model.add(LSTM(100, return_sequences=False, dropout=0.3, recurrent_dropout=0.3, 
               kernel_regularizer=l1_l2(l1=0.01, l2=0.01)))  # Add L1/L2 regularization
model.add(Dense(100, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)))  # Add L1/L2 regularization
model.add(Dense(len(target_columns), activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Fit the model with the training data
model.fit(X_train, y_train, validation_split=0.2, epochs=100, callbacks=[early_stopping])

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Convert the predictions to binary format
y_pred_bin = (y_pred > 0.5).astype(int)

# Print the classification report
print(classification_report(y_test, y_pred_bin, target_names=target_columns))

# Calculate precision, recall, and F1 score
precision = precision_score(y_test, y_pred_bin, average='weighted', zero_division=1)
recall = recall_score(y_test, y_pred_bin, average='weighted', zero_division=1)
f1 = f1_score(y_test, y_pred_bin, average='weighted', zero_division=1)

# Print precision, recall, and F1 score
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


Epoch 1/100
61/61 [==============================] - 15s 202ms/step - loss: 29.6736 - accuracy: 0.5157 - val_loss: 11.3701 - val_accuracy: 0.5288
Epoch 2/100
61/61 [==============================] - 11s 187ms/step - loss: 4.6537 - accuracy: 0.5265 - val_loss: 1.5864 - val_accuracy: 0.5288
Epoch 3/100
61/61 [==============================] - 12s 194ms/step - loss: 0.9383 - accuracy: 0.5265 - val_loss: 0.6113 - val_accuracy: 0.5288
Epoch 4/100
61/61 [==============================] - 12s 194ms/step - loss: 0.5767 - accuracy: 0.5265 - val_loss: 0.5516 - val_accuracy: 0.5288
Epoch 5/100
61/61 [==============================] - 12s 200ms/step - loss: 0.5381 - accuracy: 0.5265 - val_loss: 0.5243 - val_accuracy: 0.5288
Epoch 6/100
61/61 [==============================] - 12s 195ms/step - loss: 0.5270 - accuracy: 0.5265 - val_loss: 0.5239 - val_accuracy: 0.5288
Epoch 7/100
61/61 [==============================] - 12s 193ms/step - loss: 0.5278 - accuracy: 0.5265 - val_loss: 0.5277 - val_accurac

c:\Users\patri\.conda\envs\ml4qs_p3_8\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\patri\.conda\envs\ml4qs_p3_8\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
# Define your classifier with the best parameters
clf = RandomForestClassifier(
    n_estimators=best_params['estimator__n_estimators'],
    max_features=best_params['estimator__max_features'],
    max_depth=best_params['estimator__max_depth'],
    criterion=best_params['estimator__criterion'],
    min_samples_split=10,  # Increase this value to add more regularization
    min_samples_leaf=5,  # Increase this value to add more regularization
    random_state=42,
    class_weight='balanced'
)

# Wrap your classifier with OneVsRestClassifier
clf = OneVsRestClassifier(clf)

# Fit the model with the training data
clf.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = clf.predict(X_test)

# Print the classification report
print(classification_report(y_test, y_pred, target_names=target_columns))

# Calculate precision, recall, and F1 score
precision = precision_score(y_test, y_pred, average='weighted', zero_division=1)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=1)
f1 = f1_score(y_test, y_pred, average='weighted', zero_division=1)

# Print precision, recall, and F1 score
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


              precision    recall  f1-score   support

labelCycling       1.00      1.00      1.00       247
 labelStairs       1.00      1.00      1.00        64
labelWalking       1.00      1.00      1.00       192
labelSitting       1.00      1.00      1.00       649
  labelOther       1.00      1.00      1.00        50

   micro avg       1.00      1.00      1.00      1202
   macro avg       1.00      1.00      1.00      1202
weighted avg       1.00      1.00      1.00      1202
 samples avg       1.00      1.00      1.00      1202

Precision: 1.0
Recall: 1.0
F1 Score: 1.0


c:\Users\patri\.conda\envs\ml4qs_p3_8\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\patri\.conda\envs\ml4qs_p3_8\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
# Define the LSTM model
model = Sequential()
model.add(LSTM(200, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, input_shape=(X_train.shape[1], 1)))  # Add dropout and recurrent dropout
model.add(LSTM(100, return_sequences=False, dropout=0.2, recurrent_dropout=0.2))  # Reduce the number of units and add dropout and recurrent dropout
model.add(Dense(10, activation='relu'))
model.add(Dense(len(target_columns), activation='sigmoid'))  # Change activation function to softmax

# Compile the model
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])  # Use learning_rate instead of lr

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)  # Stop training when the validation loss has not improved for 10 epochs

# Fit the model with the training data
model.fit(X_train, y_train, validation_split=0.2, epochs=30, callbacks=[early_stopping])  # Add validation split and early stopping

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Convert the predictions to binary format
y_pred_bin = (y_pred > 0.5).astype(int)

# Print the classification report
print(classification_report(y_test, y_pred_bin, target_names=target_columns))

# Calculate precision, recall, and F1 score
precision = precision_score(y_test, y_pred_bin, average='weighted', zero_division=1)
recall = recall_score(y_test, y_pred_bin, average='weighted', zero_division=1)
f1 = f1_score(y_test, y_pred_bin, average='weighted', zero_division=1)

# Print precision, recall, and F1 score
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


Epoch 1/30
61/61 [==============================] - 29s 399ms/step - loss: 0.5518 - accuracy: 0.2164 - val_loss: 0.4937 - val_accuracy: 0.2181
Epoch 2/30
61/61 [==============================] - 24s 398ms/step - loss: 0.4600 - accuracy: 0.2303 - val_loss: 0.4038 - val_accuracy: 0.2551
Epoch 3/30
61/61 [==============================] - 25s 404ms/step - loss: 0.3753 - accuracy: 0.5064 - val_loss: 0.2937 - val_accuracy: 0.8498
Epoch 4/30
61/61 [==============================] - 25s 411ms/step - loss: 0.2815 - accuracy: 0.8027 - val_loss: 0.1744 - val_accuracy: 0.8848
Epoch 5/30
61/61 [==============================] - 24s 397ms/step - loss: 0.1736 - accuracy: 0.8434 - val_loss: 0.1102 - val_accuracy: 0.9074
Epoch 6/30
61/61 [==============================] - 25s 410ms/step - loss: 0.1308 - accuracy: 0.8697 - val_loss: 0.0949 - val_accuracy: 0.9218
Epoch 7/30
61/61 [==============================] - 26s 429ms/step - loss: 0.1188 - accuracy: 0.9037 - val_loss: 0.0750 - val_accuracy: 0.9444

c:\Users\patri\.conda\envs\ml4qs_p3_8\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\patri\.conda\envs\ml4qs_p3_8\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [35]:
from keras.callbacks import EarlyStopping

# Define a function to create and compile a new model
def create_model():
    model = Sequential()
    model.add(LSTM(200, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, input_shape=(X_train.shape[1], 1)))
    model.add(LSTM(100, return_sequences=False, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(len(target_columns), activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
    return model

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Create a new model
model = create_model()

# Fit the model with the training data
history = model.fit(X_train, y_train, validation_split=0.2, epochs=30, callbacks=[early_stopping])

# Evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)

# Print the loss and accuracy
print(f"Loss: {scores[0]}")
print(f"Accuracy: {scores[1]}")


Epoch 1/30
61/61 [==============================] - 37s 509ms/step - loss: 0.4750 - accuracy: 0.5064 - val_loss: 0.4013 - val_accuracy: 0.5597
Epoch 2/30
61/61 [==============================] - 32s 528ms/step - loss: 0.4031 - accuracy: 0.5374 - val_loss: 0.3733 - val_accuracy: 0.5597
Epoch 3/30
61/61 [==============================] - 33s 542ms/step - loss: 0.3772 - accuracy: 0.5647 - val_loss: 0.3583 - val_accuracy: 0.5597
Epoch 4/30
61/61 [==============================] - 32s 522ms/step - loss: 0.3603 - accuracy: 0.6012 - val_loss: 0.3289 - val_accuracy: 0.6955
Epoch 5/30
61/61 [==============================] - 30s 498ms/step - loss: 0.3271 - accuracy: 0.6589 - val_loss: 0.2810 - val_accuracy: 0.7675
Epoch 6/30
61/61 [==============================] - 30s 496ms/step - loss: 0.2680 - accuracy: 0.7393 - val_loss: 0.2367 - val_accuracy: 0.7757
Epoch 7/30
61/61 [==============================] - 31s 500ms/step - loss: 0.2451 - accuracy: 0.7429 - val_loss: 0.2196 - val_accuracy: 0.7819

In [38]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

# Predict the values from the validation dataset
Y_pred = model.predict(X_test)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred, axis = 1) 

# Convert validation observations to one hot vectors
Y_true = np.argmax(y_test, axis = 1) 

# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
 

# plot the confusion matrix
f,ax = plt.subplots(figsize=(8, 8))
sns.heatmap(confusion_mtx, annot=True, linewidths=0.01,cmap="Greens",linecolor="gray", fmt= '.1f',ax=ax)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

print(classification_report(Y_true, Y_pred_classes, target_names=target_columns))


38/38 [==============================] - 5s 122ms/step


ValueError: Shape of passed values is (1196, 1), indices imply (1196, 5)

In [ ]:
# Plot the accuracy curves for training and validation 
plt.plot(history.history['accuracy'], color='b', label="Training accuracy")
plt.plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
plt.title("Training session's progress over iterations")
plt.legend(loc='lower right')
plt.ylabel('Training Progress (Accuracy)')
plt.xlabel('Training Epoch')
plt.ylim(0,1)
plt.show()


In [34]:
# Make predictions on the testing data
y_pred_bin = (model.predict(X_test) > 0.5).astype(int)

# Print the classification report
print(classification_report(y_test, y_pred_bin, target_names=target_columns))

# Print precision, recall,and F1 score
print(f"Precision: {precision_score(y_test, y_pred_bin, average='weighted', zero_division=1)}")
print(f"Recall: {recall_score(y_test, y_pred_bin, average='weighted', zero_division=1)}")
print(f"F1 Score: {f1_score(y_test, y_pred_bin, average='weighted', zero_division=1)}")

AttributeError: 'KerasClassifier' object has no attribute 'model'

In [30]:
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score, KFold

# Define a function to create and compile a new model
def create_model():
    model = Sequential()
    model.add(LSTM(200, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, input_shape=(X_train.shape[1], 1)))
    model.add(LSTM(100, return_sequences=False, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(len(target_columns), activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
    return model

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Create a new model
model = KerasClassifier(build_fn=create_model, epochs=30, batch_size=10, verbose=0)

# Define the cross validation iterator
kfold = KFold(n_splits=10, shuffle=True)

# Perform cross validation
results = cross_val_score(model, X_train, y_train, cv=kfold)

# Print the cross validation score
print(f"Cross Validation Accuracy: {results.mean()} (+/- {results.std() * 2})")


C:\Users\patri\AppData\Local\Temp\ipykernel_18160\125865451.py:19: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, epochs=30, batch_size=10, verbose=0)


Cross Validation Accuracy: 0.9946417093276978 (+/- 0.013318841094162872)


In [31]:
print(results)

[1.         1.         1.         1.         0.9835391  0.98765433
 0.99588478 0.9834711  1.         0.99586779]


AttributeError: 'KerasClassifier' object has no attribute 'layers'